In [ ]:
# ! pip install plot_keras_history

In [ ]:
# !pip install git+https://www.github.com/keras-team/keras-contrib.git

In [1]:
%tensorflow_version 1.13.1

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.13.1`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [3]:
import tensorflow
print(tensorflow.__version__)

1.15.2


In [5]:
import pickle
import operator
import re
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from plot_keras_history import plot_history
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix
from keras_contrib.utils import save_load_utils


from keras import layers
from keras import optimizers

from keras.models import Model
from keras.models import Input

from keras_contrib.layers import CRF
from keras_contrib import losses
from keras_contrib import metrics

## Prepare our data

In [6]:
import pickle 
with open('mylist', 'rb') as f: 
  mylist = pickle.load(f) 

In [7]:
train_list = [x for x in mylist if len(x[1])>1]
len(train_list)

6053

In [8]:
training_data = [x[0].replace("s'"," s'").replace("'s"," 's").replace(',',' ,').replace(':',' :').replace('.',' .').split() for x in train_list]
training_data

[['Ability',
  'to',
  ':',
  'Apply',
  'sound',
  'fiscal',
  'and',
  'administrative',
  'practices',
  'to',
  'the',
  'company',
  "'s",
  'activities',
  '.'],
 ['Strong',
  'financial',
  'planning',
  'and',
  'analytical',
  'skills',
  'and',
  'experience',
  'and',
  'the',
  'ability',
  'to',
  'work',
  'closely',
  'with',
  'and',
  'support',
  'the',
  'CEO',
  'and',
  'other',
  'executives',
  'in',
  'strategic',
  'development',
  'and',
  'implementation',
  '.'],
 ['Experience',
  'with',
  'behavior',
  'change',
  'in',
  'reproductive',
  'health',
  'and/or',
  'family',
  'planning',
  'programs',
  '.'],
 ['Experience',
  'with',
  'PC-based',
  'software',
  ',',
  'including',
  'word',
  'processing',
  'and',
  'basic',
  'spreadsheet',
  'analysis',
  '.'],
 ['High',
  'level',
  'of',
  'written',
  'and',
  'spoken',
  'English',
  'fluency',
  '.'],
 ['and',
  'three',
  'years',
  'of',
  'experience',
  'as',
  'chief',
  'of',
  'party',
  '

In [9]:
label_data = []
from tqdm import tqdm
for i in tqdm(range(len(train_list))):
  cur_sample_label = []
  labels = train_list[i][1]
  for j in range(len(training_data[i])):
    if training_data[i][j] in labels:
      cur_sample_label.append('S')
    else:
      cur_sample_label.append('O')
  label_data.append(cur_sample_label)

100%|██████████| 6053/6053 [00:00<00:00, 41227.80it/s]


### take a look at our data

In [10]:
print(training_data[0])
print(label_data[0])

['Ability', 'to', ':', 'Apply', 'sound', 'fiscal', 'and', 'administrative', 'practices', 'to', 'the', 'company', "'s", 'activities', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'S', 'O', 'O', 'O', 'O', 'O', 'S', 'O']


In [11]:
#prepare word2index index2word, refer to https://blog.dominodatalab.com/named-entity-recognition-ner-challenges-and-model
import operator
import numpy as np
all_words = []
for l in training_data:
  all_words += l
all_unique_words = list(set(all_words))
all_unique_tags = ['O','S']

max_lenth = max(len(item) for item in training_data)
print(max_lenth)

word2index = {word: idx + 2 for idx, word in enumerate(all_unique_words)}
word2index["--UNKNOWN_WORD--"]=0
word2index["--PADDING--"]=1
index2word = {idx: word for word, idx in word2index.items()}

tag2index = {tag: idx + 1 for idx, tag in enumerate(all_unique_tags)}
tag2index["--PADDING--"] = 0

index2tag = {idx: word for word, idx in tag2index.items()}


123


In [ ]:

X = [[word2index[word] for word in sentence] for sentence in training_data]
y = [[tag2index[tag] for tag in sentence] for sentence in label_data]
print("X[0]:", X[0])
print("y[0]:", y[0])

X = [sentence + [word2index["--PADDING--"]] * (max_lenth - len(sentence)) for sentence in X]
y = [sentence + [tag2index["--PADDING--"]] * (max_lenth - len(sentence)) for sentence in y]
print("X[0]:", X[0])
print("y[0]:", y[0])

TAG_COUNT = len(tag2index)
y_onehot = [np.eye(TAG_COUNT)[sentence] for sentence in y]
print("y_onehot[0]:", y_onehot[0])

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.1, random_state=1234)

print("Number of sentences in the training dataset: {}".format(len(X_train)))
print("Number of sentences in the test dataset : {}".format(len(X_test)))

Number of sentences in the training dataset: 5447
Number of sentences in the test dataset : 606


In [46]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

## LSTM - CRF Model

In [16]:
WORD_COUNT = len(index2word)
DENSE_EMBEDDING = 100
LSTM_UNITS = 50
LSTM_DROPOUT = 0.1
DENSE_UNITS = 100
BATCH_SIZE = 60
MAX_EPOCHS = 5

In [ ]:
#download embedding
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

In [18]:
import os
import pathlib
# path_to_glove_file = os.path.join(
#     os.path.expanduser("~"), ".keras/datasets/glove.6B.100d.txt"
# )

embeddings_index = {}
with open('glove.6B.100d.txt') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [20]:
#Read GloVe embeddings.
# def read_GloVe(filename):
#     embeddings = {}
#     for line in open(filename).readlines():
#         #print(line)
#         fields = line.strip().split(" ")
#         word = fields[0]
#         embeddings[word] = [float(x) for x in fields[1:]]
#     return embeddings

# GloVe = read_GloVe("glove.840B.300d.conll_filtered.txt")

# print(GloVe["the"])
# print("dimension of glove embedding:", len(GloVe["the"]))

num_tokens = WORD_COUNT
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word2index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        embedding_matrix[i] = np.random.normal(0,1,100)
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 3261 words (2325 misses)


In [21]:
from tensorflow.keras.layers import Embedding
import keras

In [36]:
import tensorflow
from keras.optimizers import adam
input_layer = layers.Input(shape=(max_lenth,))

# model = layers.Embedding(
#     WORD_COUNT,
#     DENSE_EMBEDDING,
#     embeddings_initializer=keras.initializers.Constant(embedding_matrix),
#     trainable=True,
#     input_length=max_lenth
# )(input_layer)

model = layers.Embedding(WORD_COUNT, DENSE_EMBEDDING, embeddings_initializer="uniform", input_length=max_lenth)(input_layer)

model = layers.Bidirectional(layers.LSTM(LSTM_UNITS, recurrent_dropout=LSTM_DROPOUT, return_sequences=True))(model)

model = layers.TimeDistributed(layers.Dense(DENSE_UNITS, activation="relu"))(model)

# crf_layer = CRF(units=TAG_COUNT)
crf_layer = layers.Dense(TAG_COUNT, activation="relu")
output_layer = crf_layer(model)

ner_model = Model(input_layer, output_layer)

# loss = losses.crf_loss
# acc_metric = metrics.crf_accuracy
opt = adam(lr=0.001)
ner_model.compile(optimizer=opt, loss=tensorflow.keras.losses.BinaryCrossentropy(), metrics=[tensorflow.keras.metrics.AUC()])
# ner_model.compile(optimizer=opt, loss=loss, metrics=[acc_metric])

ner_model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 123)               0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 123, 100)          558600    
_________________________________________________________________
bidirectional_6 (Bidirection (None, 123, 100)          60400     
_________________________________________________________________
time_distributed_6 (TimeDist (None, 123, 100)          10100     
_________________________________________________________________
dense_9 (Dense)              (None, 123, 3)            303       
Total params: 629,403
Trainable params: 629,403
Non-trainable params: 0
_________________________________________________________________


In [43]:
np.array(y_onehot).shape

(6053, 123, 3)

In [48]:
history = ner_model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=MAX_EPOCHS, validation_split=0.1, verbose=2)

Train on 4902 samples, validate on 545 samples
Epoch 1/5
 - 22s - loss: 0.2124 - auc: 0.9638 - val_loss: 0.0499 - val_auc: 0.9821
Epoch 2/5
 - 21s - loss: 0.0479 - auc: 0.9880 - val_loss: 0.0421 - val_auc: 0.9914
Epoch 3/5
 - 21s - loss: 0.0384 - auc: 0.9931 - val_loss: 0.0303 - val_auc: 0.9944
Epoch 4/5
 - 22s - loss: 0.0273 - auc: 0.9953 - val_loss: 0.0243 - val_auc: 0.9960
Epoch 5/5
 - 22s - loss: 0.0217 - auc: 0.9965 - val_loss: 0.0231 - val_auc: 0.9969


In [51]:
y_pred = ner_model.predict(X_test)

y_pred = np.argmax(y_pred, axis=2)

y_test = np.argmax(y_test, axis=2)

accuracy = (y_pred == y_test).mean()
print(accuracy)

0.986449864498645


In [52]:
def tag_conf_matrix(cm, tagid):
    tag_name = index2tag[tagid]
    print("Tag name: {}".format(tag_name))
    print(cm[tagid])
    tn, fp, fn, tp = cm[tagid].ravel()
    tag_acc = (tp + tn) / (tn + fp + fn + tp)
    print("Tag accuracy: {:.3f} \n".format(tag_acc))

matrix = multilabel_confusion_matrix(y_test.flatten(), y_pred.flatten())

tag_conf_matrix(matrix, 0)
tag_conf_matrix(matrix, 1)
tag_conf_matrix(matrix, 2)

Tag name: --PADDING--
[[10892     0]
 [    0 63646]]
Tag accuracy: 1.000 

Tag name: O
[[64925   721]
 [  289  8603]]
Tag accuracy: 0.986 

Tag name: S
[[72249   289]
 [  721  1279]]
Tag accuracy: 0.986 



In [58]:
ind = 90

print([index2word[word] for word in X_test[ind]])
print([index2tag[tag] for tag in y_pred[ind]])
print([index2tag[tag] for tag in y_test[ind]])

['At', 'least', '2', 'years', 'of', 'experience', 'with', 'Java', ',', 'PHP', 'or', 'C++', '.', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--PADDING--', '--P